In [2]:
# given a topology and msa, optimize the branch lengths
# then calculate the likelihood

In [79]:
import numpy as np

# test data
t_C = '[&R] ((1,2),(3,4));' # no branches
# t_C = '[&R] ((1,2), 3);'
# species by 3-site matrix
msa = np.array([[1,0,2], # species 1
                 [1,0,0], # species 2
                 [0,1,2], # species 3
                 [0,1,0]] # species 4
               )

In [139]:
seq_dict = dict()
seq_dict[1] = [1, 0, 2]
seq_dict[2] = [1, 0, 0]
seq_dict[3] = [0, 1, 2]
seq_dict[4] = [0, 1, 0]

In [81]:
# estimate distance matrix first
distmat = dict()
for a in seq_dict.keys():
    for b in seq_dict.keys():
        distmat[(a, b)] = sum(est_d(seq_dict, a, b))

In [171]:
# estimate the length of r -> r* for each internal node r

In [129]:
def sets(seq_dict, a, b):
    # get the msa
    seq_a = np.array(seq_dict[a])
    seq_b = np.array(seq_dict[b])
    
    k = len(seq_a)
    
    ## calculate the sets  
    s_0, s_1a, s_1b, s_2, s3 = set(), set(), set(), set(), set()
    for idx in range(seq_a):
        c_a, c_b = seq_a[idx], seq_b[idx]
        if c_a == c_b:
            if c_a == 0:
                s_0.add(idx)
            else:
                s_2.add(idx)
        elif c_a == 0: # then c_b != 0
            s_1b.add(idx)
        elif c_b == 0: # then c_a != 0
            s_1a.add(idx)
        else:
            s_3.add(idx)
    
    assert len(s_0) + len(s_1a) + len(s_1b) + len(s_2) + len(s3) == k
    
    sets = dict()
    sets['s_0'] = s_0
    sets['s_1a'] = s_1a
    sets['s_1b'] = s_1b
    sets['s_2'] = s_2
    sets['s_3'] = s_3
    
    return sets


In [131]:
def get_taxon_name(node):
    return int(node.taxon.__str__().replace("'", ""))

In [158]:
def est_d(seq_a, seq_b):
#     if type(a) == int:
#         seq_a, seq_b = seq_dict[a], seq_dict[b]
#     else:
#         a_name = get_taxon_name(a)
#         b_name = get_taxon_name(b)
#         seq_a, seq_b = seq_dict[a_name], seq_dict[b_name]
    
    # leaf nodes a and b
    Z_a = seq_a.count(0)
    Z_b = seq_b.count(0)
    Z_ab = set()
    
    for i in range(len(seq_a)):
        a_i, b_i = seq_a[i], seq_b[i]
        if a_i == 0:
            Z_ab.add(i)
        if b_i == 0:
            Z_ab.add(i)
        if a_i != b_i and a_i != 0 and b_i != 0:
            Z_ab.add(i)
    Z_ab = len(Z_ab)
    
    d_a = - np.log(Z_a/Z_ab)
    d_b = - np.log(Z_b/Z_ab)
    
    if d_a == -0.0:
        d_a = 0.0
    if d_b == -0.0:
        d_b = 0.0
        
    return d_a, d_b

In [155]:
# q_ialpha
def transition(alpha, site, numsites):
    # transition rate 0 -> char alpha at site i
    lambda_ialpha = np.sum(msa.T[site] == alpha) / len(msa.T[site])
    lambda_all = np.sum(msa.T[site] != 0) / len(msa.T[site])
    return lambda_ialpha / lambda_all

In [166]:
def parsimonious(seq_a, seq_b): 
    # get the most parsimonious ancestral sequence to explain both these children
    k = len(seq_a)
    anc_seq = []
    for idx in range(k):
        c_a = seq_a[idx]
        c_b = seq_b[idx]
        
        if c_a == c_b: # (0, 0) or (1, 1)
            anc_seq.append(c_a)
        elif c_a == 0 or c_b == 0: # (0, 1) or (1, 0)
            anc_seq.append(0)
        elif c_a != c_b: # (1, 2)
            anc_seq.append(0)
    return anc_seq
        

In [170]:
# fill in the parsimony ancestral sequences and then estimate each branch length
import dendropy
import itertools

# assume binary tree!
nwkt = dendropy.Tree.get(data=t_C, schema="newick")
print(nwkt)
# fill in the edges that I know so far
for n in nwkt.postorder_node_iter():
    if n.is_internal():
        a, b = n.child_nodes()
        if a.is_leaf() and b.is_leaf():
            taxon_a = get_taxon_name(a)
            taxon_b = get_taxon_name(b)
            seq_a = seq_dict[taxon_a]
            seq_b = seq_dict[taxon_b]
        
            e_left, e_right = est_d(seq_a, seq_b)
            a.edge.length = e_left
            b.edge.length = e_right
            
            a.label = seq_a
            b.label = seq_b
            n.label = parsimonious(seq_a, seq_b)
            
        elif a.is_leaf(): 
            # get b's parsimonious sequence
            e_left, e_right = est_d(seq_dict[a], b.label)
            a.edge.length = e_left
            b.edge.length = e_right
            
            a.label = seq_a
            b.label = b.label
            n.label = parsimonious(seq_a, b.label)
            
        elif b.is_leaf():
            # print("a is internal, b is leaf")
            e_left, e_right = est_d(a.label, b)
            a.edge.length = e_left
            b.edge.length = e_right
            
            a.label = a.label
            b.label = seq_b
            n.label = parsimonious(a.label, seq_b)
            
        else: # both internal
            e_left, e_right = est_d(a.label, b.label)
            a.edge.length = e_left
            b.edge.length = e_right
            
            a.label = a.label
            b.label = b.label
            n.label = parsimonious(a.label, b.label)

e0 = []
for e in nwkt.postorder_edge_iter():
    e0.add(e.length)


((1,2),(3,4))
0.6931471805599453
0.0
0.40546510810816444
0.6931471805599453
0.0
0.40546510810816444
None


In [172]:
sets

<function __main__.sets(seq_dict, a, b)>

In [ ]:
# Optimize the branch lengths w/ marginal likelihood, using scipy.optimize
k = 3

def likelihood(k, d_a, d_b, d_r, sets, q_ialpha=0.2): 
    s_0 = len(sets['s_0'])
    s_1a = len(sets['s_1a'])
    s_1b = len(sets['s_1b'])
    s_2 = len(sets['s_2'])
    s_3 = len(sets['s_3'])
    
    p1 = -(s_1b + s_0) * d_a + (s_1a + s_3) * np.log(1 - math.exp(-d_a))
    p2 = -(s_1a + s_0) * d_b + (s_1b + s_3) * np.log(1 - math.exp(-d_b)) - (k - s_2) * d_r
    p3 = 0.0
    
    for i in range(s_2):
        # q_ialpha is the transition rate at site i from 0 -> character at node a at site i
        # iterate over sites in s_2
        p3 += np.log(q_ialpha^2 * (1 - math.exp(-d_a)) * (1 - math.exp(-d_b)) * math.exp(-d_r) + q_ialpha*(1 - math.exp(-d_r)))
    
    return p1 + p2 + p3


In [173]:
scipy.optimize.minimize(likelihood, e0, method="SLSQP", tol=1e2, options={'disp': True}, constraints=cons )


NameError: name 'e0' is not defined

In [ ]:
# # Optimize the branch lengths w/ marginal likelihood, using scipy.optimize

# def likelihood(k, d_a, d_b, d_r, sets, q_ialpha=0.2): 
#     s_0 = len(sets['s_0'])
#     s_1a = len(sets['s_1a'])
#     s_1b = len(sets['s_1b'])
#     s_2 = len(sets['s_2'])
#     s_3 = len(sets['s_3'])
    
#     p1 = -(s_1b + s_0) * d_a + (s_1a + s_3) * np.log(1 - math.exp(-d_a))
#     p2 = -(s_1a + s_0) * d_b + (s_1b + s_3) * np.log(1 - math.exp(-d_b)) - (k - s_2) * d_r
#     p3 = 0.0
    
#     for i in range(s_2):
#         # q_ialpha is the transition rate at site i from 0 -> character at node a at site i
#         # iterate over sites in s_2
#         p3 += np.log(q_ialpha^2 * (1 - math.exp(-d_a)) * (1 - math.exp(-d_b)) * math.exp(-d_r) + q_ialpha*(1 - math.exp(-d_r)))
    
#     return p1 + p2 + p3


In [152]:
# # let's just iterate over all the internal branches one at a time first
# import dendropy
# import itertools

# # assume binary tree!
# nwkt = dendropy.Tree.get(data=t_C, schema="newick")
# print(nwkt)
# # fill in the edges that I know so far
# for n in nwkt.postorder_node_iter():
#     # print(n)
#     if n.is_internal():
#         a, b = n.child_nodes()
#         if a.is_leaf() and b.is_leaf():
#             e_left, e_right = est_d(seq_dict, a, b)
#             a.edge.length = e_left
#             b.edge.length = e_right
#         elif a.is_leaf(): # b is internal
#             e_left, e_right = est_d(seq_dict, a, b.leaf_nodes()[0])
#             # print("a is leaf, b is internal")
#             a.edge.length = e_left
#             b.edge.length = e_right - b.leaf_nodes()[0].edge.length
#         elif b.is_leaf():
#             # print("a is internal, b is leaf")
#             e_left, e_right = est_d(seq_dict, a.leaf_nodes()[0], b)
#             a.edge.length = e_left - a.leaf_nodes()[0].edge.length
#             b.edge.length = e_right
#         else: # both internal
#             a1, a2 = a.child_nodes()
#             b1, b2 = b.child_nodes()
            
#             a1_b1 = sum(est_d(seq_dict, a1, b1))
#             a2_b2 = sum(est_d(seq_dict, a2, b2))
#             a1_a2 = sum(est_d(seq_dict, a1, a2))
#             b1_b2 = sum(est_d(seq_dict, b1, b2))
            
#             internal_edge = (a1_b1 + a2_b2 - a1_a2 - b1_b2)/(2*2) # root at half the internal edge
#             a.edge.length = internal_edge
#             b.edge.length = internal_edge
# print(nwkt)

# # how does iqtree, raxml or fasttree know when to stop opt branch lengths?
# # do they calculate the tree likelihood?


In [126]:
# now let's optimize the branches
import math
from scipy import optimize

def l(x):
    d_a, d_b, d_r = x
    print("d_a", d_a, "d_b", d_b, "d_r", d_r)
    q = 0.2
    
    k = 1000
    s_1a = 100
    s_1b = 100
    s_0 = 600 
    s_2 = 100 
    s_3 = 100
    
    p1 = -(s_1b + s_0) * d_a + (s_1a + s_3) * np.log(1 - math.exp(-d_a))
    p2 = -(s_1a + s_0) * d_b + (s_1b + s_3) * np.log(1 - math.exp(-d_b)) - (k - s_2) * d_r
    p3 = 0.0
    
    for i in range(s_2):
        p3 += np.log(q**2 * (1 - math.exp(-d_a)) * (1 - math.exp(-d_b)) * math.exp(-d_r) + q*(1 - math.exp(-d_r)))
    
    print(p1, p2, p3, "sum", p1+p2+p3)
    return p1 + p2 + p3

def con(t):
    return t

cons = [{'type': 'ineq', 'fun': con}]
# for e in nwkt.postorder_edge_iter():
x0 = (0.05,0.05,0.001) # [k, d_a, d_b, d_r, sets, q_ialpha]
a = scipy.optimize.minimize(l, x0, method="SLSQP", tol=1e2, options={'disp': True}, constraints=cons )

d_a 0.05 d_b 0.05 d_r 0.001
-639.1256218114754 -640.0256218114754 -812.8712507171285 sum -2092.0224943400794
d_a 0.0500000149011612 d_b 0.05 d_r 0.001
-639.1255741153515 -640.0256218114754 -812.8712413513507 sum -2092.0224372781777
d_a 0.05 d_b 0.0500000149011612 d_r 0.001
-639.1256218114754 -640.0255741153514 -812.8712413513507 sum -2092.0224372781777
d_a 0.05 d_b 0.05 d_r 0.0010000149011611939
-639.1256218114754 -640.0256352225205 -812.8702417849817 sum -2092.0214988189778
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2092.0224943400794
            Iterations: 5
            Function evaluations: 4
            Gradient evaluations: 1


In [127]:
a

     fun: -2092.0224943400794
     jac: array([ 3829.35940552,  3829.35940552, 66808.29022217])
 message: 'Optimization terminated successfully'
    nfev: 4
     nit: 5
    njev: 1
  status: 0
 success: True
       x: array([0.05 , 0.05 , 0.001])

In [101]:
# SLSQP - 

# put the whole felsenstein's in, colalpse the vector 
# write the function for three 

In [114]:
1 - math.exp()

0.048770575499285984

In [122]:
np.log(1 - math.exp(0.05))

/Users/gillianchu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


nan

In [123]:
1 - math.exp(0.05)

-0.05127109637602412